In [4]:
# 导入必要的库
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords

papers = []
try:
    with open('train.json', 'r',encoding='utf-8',errors='ignore') as train:
        
        for line in train.readlines():
            data_train = json.loads(line)
            papers.append(data_train)
    
except FileNotFoundError:
    print("File not found")

In [1]:
import spacy

In [2]:
nlp = spacy.load('en_core_web_sm')
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [5]:
texts_train = []
labels_train = []

for i in papers[:int(len(papers)/2)]:
    for j in i['entityMentions']:
        texts_train.append(j['text'])
        labels_train.append(j['label'])
print(texts_train)
print(labels_train)

['林某某', '海洛因', '2014年8月7日早上7时', '徐某某', '林某某', '徐某某', '林某某', '0.07克', '0.44克', '海洛因', '2015年7月22日1时', '吴某某', '贵阳市云岩区', '焦某某', '海洛因', '120克', '2014年12月', '黄某某', '博罗县罗阳镇田牌村', '氯胺酮', '黄某明', '刘某林', '刘某强', '2015年2月份', '黄某某', '博罗县罗阳镇田牌村', '甲基苯丙胺', '冰毒', '胡某康', '杨某宇', '10月24日', '博罗县罗阳镇', '黄某某', '2016年1月26日2时', '腾冲市腾越镇', '吴旭', '甲基苯丙胺', '157.9克', '当日15时', '吴旭', '2014年4月9日', '纳某某', '永宁县东泰阳光大酒店楼下的一辆黑色别克轿车上', '甲基苯丙胺', '冰毒', '1克', '吕某某', '2014年7、8月间某日晚', '颜某', '扬州市邗江区文汇苑B区365超市门口的公交站台', '黄某', '甲基苯丙胺', '0.2克', '2014年10月23日22时', '重庆市沙坪坝区沙正街', '殷小伟', '殷小伟背', '甲基苯丙胺', '冰毒', '104.98克', '甲基苯丙胺', '1.02克', '2014年6月份、8月份期间', '赵某某', '马某', '阜南县鹿城镇焦阳路97号家中', '赵某某', '2014年2月至2014年6月18日间', '颜某', '龙海市浮宫镇霞郭村', '洪某', '林某', '曾某', '甲基苯丙胺', '冰毒', '洪某', '林某', '曾某', '2014年9月25日18时', '陆某某', '黎XX', '甲基苯丙胺', '贵港市港北区XX街XX网吧门口', '甲基苯丙胺', '黎XX', '陆某某', '黎XX', '甲基苯丙胺', '0.79克', '2014年10月13日22时', '雷某', '申某某', '王某某', '资中县水南镇中央大街', '甲基苯丙胺', '饶某', '饶某', '0.23克', '甲基苯丙胺', '2013年12月26日凌晨4时', '南昌市西湖区桃花镇', '喻某', '喻某', '冰毒', '8.8克', '麻古

In [6]:
texts_train = [preprocess(text) for text in texts_train]
texts_test = texts_train[:int(len(texts_train)/2)]

In [7]:
tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(texts_train)
X_test_tfidf = tfidf_vectorizer.transform(texts_test)

In [8]:
nb_model = MultinomialNB()

nb_model.fit(X_train_tfidf, labels_train)

MultinomialNB()

In [9]:
labels_pred = nb_model.predict(X_test_tfidf)

In [10]:
labels_test = labels_train[:int(len(labels_train)/2)]
print("Accuracy:", accuracy_score(labels_test, labels_pred))
print(classification_report(labels_test, labels_pred))

Accuracy: 0.8942961800104657
              precision    recall  f1-score   support

         NDR       1.00      1.00      1.00       831
          NT       1.00      0.99      1.00       541
          NW       0.98      0.92      0.95       401
          Nh       0.80      1.00      0.89      1610
          Ns       1.00      0.16      0.28       439

    accuracy                           0.89      3822
   macro avg       0.96      0.82      0.82      3822
weighted avg       0.91      0.89      0.87      3822



In [11]:
def extract_entities(text):
    doc = nlp(text)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

entities_train = [extract_entities(text) for text in texts_train]
entities_test = [extract_entities(text) for text in texts_test]

In [12]:
def extract_relationships(text):
    doc = nlp(text)
    relationships = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            for token in doc:
                if token.dep_ == 'nsubj' and token.head.pos_ == 'VERB':
                    relationships.append((ent.text, token.text, token.head.text))
                    return relationships

relationships_train = [extract_relationships(text) for text in texts_train]
relationships_test = [extract_relationships(text) for text in texts_test]

In [13]:
entity_df_train = pd.DataFrame({'text': texts_train, 'entities': entities_train})
entity_df_test = pd.DataFrame({'text': texts_test, 'entities': entities_test})

relationship_df_train = pd.DataFrame({'text': texts_train, 'relationships': relationships_train})
relationship_df_test = pd.DataFrame({'text': texts_test, 'relationships': relationships_test})

In [14]:
entity_df_train.to_json('entities_train.json', orient='records', force_ascii=False)
entity_df_test.to_json('entities_test.json', orient='records', force_ascii=False)
relationship_df_train.to_json('relationships_train.json', orient='records', force_ascii=False)
relationship_df_test.to_json('relationships_test.json', orient='records', force_ascii=False)